<h3> 1. ﻿naver finance에서 코스피 200의 리스트를 받아라</h3>

In [1]:
!pip install pandas-ta

In [51]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import pandas_ta as pta
import re
import numpy as np

In [17]:
jongmoks = [] #코스피200의 종목이름
codes = [] #코스피200의 종목코드
for page in range(1,21):
    url = f'https://finance.naver.com/sise/entryJongmok.nhn?&page={page}'
    html = rq.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    items = soup.find_all('td', {'class': 'ctg'})
    
    for item in items:
        txt = item.a.get('href') #item에서 <a href =''>부분만 추출
        k = re.search('[\d]+', txt) #href에 포함된 종목코드만 추출, 문자열 안에서 정수만 추출
        if k:
            code = k.group()
            codes.append(code)
            name = item.text
            jongmoks.append(name)
    #출처: https://titanism-ebot.tistory.com/3
        
data = dict(zip(codes, jongmoks)) #종목코드와 종목이름을 결합한 딕셔너리

005930
373220
207940
000660
051910
006400
005380
035420
000270
035720
068270
005490
028260
105560
012330
055550
003670
096770
034730
066570
032830
003550
033780
086790
015760
323410
010130
259960
034020
017670
011200
009830
329180
009150
036570
010950
051900
018260
030200
000810
003490
316140
024110
377300
011070
090430
086280
302440
011170
352820
097950
326030
383220
009540
402340
032640
034220
035250
028050
000060
267250
018880
138040
010140
271560
047810
361610
000720
251270
004020
011780
078930
000100
021240
161390
005830
011790
006800
241560
029780
012450
008560
010620
282330
137310
004990
112610
064350
036460
071050
128940
005940
016360
007070
003410
047050
020150
008770
030000
028670
180640
001450
139480
002790
012750
006040
039490
052690
023530
014680
138930
336260
006260
010060
001040
008930
204320
272210
026960
000880
111770
001440
002380
000990
000120
042660
004170
047040
079550
006360
081660
088350
004370
000080
298050
007310
010120
069620
000150
011210
004000
285130
375500

<h3> 2. 2022년 코스피 200의 1월 ~3월까지의 주가를 모두 받아라﻿ </h3>

In [58]:
for code in data.keys():
    df = None #종목마다 다른 데이터프레임을 만들어야하기 때문에 반복할 때마다 초기화
    dic1 = {'Date':[],
       'Price':[]} #일별 종가를 저장하기 위한 딕셔너리
    
    for page in range(17, 24):
        req = rq.get(f'https://finance.naver.com/item/sise_day.naver?code={code}&page={page}', headers={'User-Agent':'Mozilla/5.0'})
        html = BeautifulSoup(req.text, 'html.parser')
        lines = html.find_all('tr', {'onmouseover':'mouseOver(this)'})
        
        for line in lines:
            date = line.find_all('td', {'align': 'center'})[0] #공휴일 제외 일자 추출
            price = line.find_all('td', {'class': 'num'})[0] #일자에 해당하는 종가 추출
            date_ = date.text.strip() #str 형식을 datetime 형식으로 변환할 때를 대비해서 공백 제거
            price_ = price.text.strip() #str 형식을 int 형식으로 변환할 때를 대비해서 공백 제거
            if date_ == "":  #date가 존재하지 않으면 dic1 딕셔너리에 일자와 종가를 추가하지 않음
                break
            if date_ not in dic1['Date']: #LG에너지솔루션의 경우 상장일인 1월 27일이 여러 번 나옴 -> 중복 제거
                dic1['Date'].append(date_)
                dic1['Price'].append(price_)
    
    df = pd.DataFrame(dic1) #해당하는 종목의 일별 종가를 담은 딕셔너리를 데이터프레임화
    df.loc[:,'Date'] = pd.to_datetime(df.loc[:,'Date'])
    df.loc[:,'Price'] = df.loc[:,'Price'].str.replace(",","")
    #출처: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html
    df.loc[:,'Price'] = pd.to_numeric(df.loc[:,'Price']) #뒤에서 연산을 위해 문자를 숫자로 바꿔준다.
    df1 = df[(df.loc[:,'Date'] >= '2022-01-01') & (df.loc[:,'Date'] <= '2022-03-31')]
    df2 = df1.sort_values(by='Date')
    #출처: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
    globals()['df_{}'.format(code)] = df2.reset_index(drop = True)
    

<h3> 3. 그들을 DataFrame화 하여 각각에 대해 기술적 분석, MA, MACD, RSI를 구하라 </h3>

In [59]:
for code in data.keys():
    # 1월부터 3월까지 공휴일이 아닌 일 수는 58일이기 때문에 5일, 20일에 대한 이동평균만 구한다.
    #단순이동평균 SMA
    globals()['df_{}'.format(code)]['Sma5'] = globals()['df_{}'.format(code)]['Price'].rolling(5).mean()
    globals()['df_{}'.format(code)]['Sma20'] = globals()['df_{}'.format(code)]['Price'].rolling(20).mean()
    #출처: https://wikidocs.net/4374
    
    #지수이동평균 EMA
    globals()['df_{}'.format(code)]['Ema5'] = globals()['df_{}'.format(code)]['Price'].ewm(5).mean()
    globals()['df_{}'.format(code)]['Ema20'] = globals()['df_{}'.format(code)]['Price'].ewm(20).mean()
    
    #가중이동평균 WMA
    def weighted_mean(weight_array):
        def inner(x):
            return (weight_array * x).mean()
        return inner
    
    weights = np.arange(1,6)
    wma5 = globals()['df_{}'.format(code)]['Price'].rolling(5).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)

    weights = np.arange(1,21)
    wma20 = globals()['df_{}'.format(code)]['Price'].rolling(20).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)

    globals()['df_{}'.format(code)]['Wma5'] = wma5
    globals()['df_{}'.format(code)]['Wma20'] = wma20
    #출처: https://wendys.tistory.com/178
    
    #MACD, MACD Signal
    exp1 = globals()['df_{}'.format(code)]['Price'].ewm(span=12, adjust=False).mean()
    exp2 = globals()['df_{}'.format(code)]['Price'].ewm(span=26, adjust=False).mean()
    globals()['df_{}'.format(code)]['MACD'] = exp1 - exp2
    globals()['df_{}'.format(code)]['Signal line'] = globals()['df_{}'.format(code)]['MACD'].ewm(span=9, adjust=False).mean()
    #출처: https://www.learnpythonwithrune.org/calucate-macd-with-pandas-dataframes/
    
    #RSI
    globals()['df_{}'.format(code)]['Rsi14'] = pta.rsi(globals()['df_{}'.format(code)]['Price'], length = 14)
    #출처: https://www.roelpeters.be/many-ways-to-calculate-the-rsi-in-python-pandas/

<h3> 4. 이들을 csv로 만들어서 코드와 csv 200개를 알집으로 제출해라</h3>

In [61]:
for code in codes:
    globals()['df_{}'.format(code)].to_csv(f"./csv/{code}.csv", sep = ",", index = False, mode= "w",encoding='cp949')
    
#출처: https://www.digitalocean.com/community/tutorials/pandas-to_csv-convert-dataframe-to-csv